<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
</b> Importing the key modules and set parameters

</div>

In [1]:
from meshparty import trimesh_io, trimesh_vtk
from caveclient import CAVEclient
import numpy as np
import pandas as pd
import cloudvolume
import itkwidgets


#setting up the cave client, if you have not done this before checkout the notebook CAVEsetup.ipynb at 
#https://github.com/AllenInstitute/MicronsBinder/tree/master/notebooks/mm3_intro
client = CAVEclient()
#client.auth.get_new_token()
#client.auth.save_token(token="Your token here")


dataset_name = 'minnie65_public_v117'
client = CAVEclient(dataset_name)


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
</b> Using the CAVEclient, we can query the synapse table for any neuron in the volume
      </div>

In [2]:
client.materialize.get_tables()

['nucleus_detection_v0',
 'synapses_pni_2',
 'nucleus_neuron_svm',
 'proofreading_status_public_release',
 'func_unit_em_match_release',
 'allen_soma_ei_class_model_v1',
 'allen_visp_column_soma_coarse_types_v1']

In [3]:
#selecting for only the proofread cells with extended axons  
proofread=client.materialize.query_table('proofreading_status_public_release', 
                                      filter_equal_dict={'status_axon':'extended'})
proofread.head()


,id,valid,pt_supervoxel_id,pt_root_id,valid_id,status_dendrite,status_axon,pt_position
0,346,t,89660913577302088,864691136194248918,864691136194248918,extended,extended,"[181147, 143788, 22167]"
1,347,t,89941288908184714,864691135517422218,864691135517422218,extended,extended,"[182993, 135610, 21130]"
2,348,t,85369450370657325,864691135753932237,864691135753932237,extended,extended,"[149711, 151738, 17565]"
3,349,t,92129866602123648,864691135367058169,864691135367058169,extended,extended,"[198949, 188854, 19772]"
4,350,t,88815663879226337,864691135293126156,864691135293126156,extended,extended,"[174921, 137614, 21123]"


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
</b> We can go on to find the post_synaptic cell id (post_pt_root_id) with the most synapses. (Hint, we recommend focusing on the proofread cells)
        </div>

In [4]:
#synapse table with all the synapses onto the proofread cells
syn_df = client.materialize.query_table('synapses_pni_2',
                                        filter_in_dict={'pre_pt_root_id': proofread.pt_root_id.values})

#subselecting only synapses onto proofread cells
syn_df['syn_num']=syn_df.groupby('post_pt_root_id')['id'].transform(len)

#which proofread cell id has the largest number of post-synaptic synapses
most_syn = max(syn_df.syn_num) 

#since there may be multiple cells with the same maximum number, we will just take the first one
cellid = syn_df[syn_df.syn_num==most_syn]['post_pt_root_id'].values[0] 


#segmentation ID for the proofread cell with the greatest number of synapses
print('Cell ID: %d, Number of synapses: %d'%(cellid,most_syn))


Cell ID: 864691135782544435, Number of synapses: 57


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
</b> Now we can move on to visualize this cell in 3D with some of it's biggest synapses
        </div>

In [5]:
seg_source = client.info.segmentation_source()

#using the google cloud segmentation path
cv = cloudvolume.CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg", use_https=True)

#you can also try the aws segmentation path
#cv = cloudvolume.CloudVolume("precomputed://s3://bossdb-open-data/microns/minnie/minnie65-flat-seg/", use_https=True)

mesh = cv.mesh.get(cellid,lod=3)[cellid]
poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data])



In [6]:
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [7]:
post_synapse_df = client.materialize.synapse_query(post_ids = np.array([cellid]))
biggest_synapses = post_synapse_df.sort_values(by=['size'],ascending=False).head(10)
print(biggest_synapses[['pre_pt_root_id', 'size']])


          pre_pt_root_id   size
7534  864691135918302748  73788
7292  864691135540444146  72048
4135  864691135531745266  67500
4397  864691135544160498  66732
2382  864691135532446450  66112
7307  864691135700690068  65264
3565  864691135417946193  64480
4577  864691135767005373  64204
8077  864691135423389454  63744
810   864691135860490863  63344


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
</b> Lastly we will visualize the cell and these 10 synapses, please note that you need to increase the size of the synapses using the slider in the widget tool
        </div>

In [12]:
#mm = trimesh_io.MeshMeta(disk_cache_path='test/test_files')
#mesh = mm.mesh(filename ='/data/dynamic_brain_workshop/electron_microscopy/2021/meshes/%d.h5'%cellid)
mesh_poly =trimesh_vtk.trimesh_to_vtk(mesh.vertices,mesh.faces,None)

poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

voxel_resolution = np.array([4,4,40])#setting the voxel resolution to convert to nm
syn_pts = np.vstack(biggest_synapses['ctr_pt_position'].values) * voxel_resolution
syn_sizes = biggest_synapses['size']
syn_actors = trimesh_vtk.point_cloud_actor(syn_pts, size=syn_sizes.values)
viewer2 =itkwidgets.view(geometries=[poly_data],
                        point_sets = [syn_pts])



In [13]:
#when visualizing the synapses, they will initially appear very small, be sure to increase the size of the 
#synapses using the sliding bar under Point Set 0. You can also change the color using color map in the widget
#You should be able to see a pyramidal neuron with 10 points indicating where it receives its 10 largest synapses
viewer2

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…